# Python notebook for finding peaks in csv file data.
Assumes folder directory structure:
<pre><code>  IMAGING
    image_stacks
    notebooks
    results
</code></pre>
NOTE: Looks for data files and stores peaks in the results directory.<br>
Execute the code sequentially, one block at a time, using &lt;shift-return&gt;.

In [ ]:
from __future__ import print_function
import glob
from ipywidgets import interact, Layout
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from scipy import signal


#### Select a data file and a data column.
NOTE: Assumes file format in which the first column contains time values in seconds and subsequent columns contain intensity data.

In [ ]:
%matplotlib widget

# global variables
data_file = ""  # the selected data file
data_col = 0    # the data column

# create data files widget
data_files = sorted([f.split('/')[-1] for f in glob.glob("../results/*.csv", recursive=False)], key=str.casefold)
data_files_widget = widgets.Select(options=data_files, description='Data file', 
                            disabled=False, layout={'width':'400px'}, style={'description_width':'100px'})
# create data columns widget
columns_widget = widgets.Select(options=[], description='Data column', 
                            disabled=False, layout={'width':'150px'}, style={'description_width':'100px'})
# create status widget
status_widget = widgets.HTML(value=' ', description=' ')

# update the columns based on the data file selection
def update_columns(*args):
  if data_files_widget.value == None:
    return
  with open("../results/" + data_files_widget.value, 'r', encoding='utf-8-sig') as f: 
    cols = (np.genfromtxt(f, dtype=float, delimiter=',')).shape[1]
  columns_widget.options = range(1,cols)

# widget change callbacks
data_files_widget.observe(update_columns, 'value')

# display and respond to the widgets
update_columns()
def f(w1, w2, w3):
  global data_file, data_col
  data_file = data_files_widget.value
  data_col = columns_widget.value

  if not data_file:
    status_widget.value = "No data file selected."
  else:
    if not data_col:
      status_widget.value = "No data column selected."
    else:
      status_widget.value = "Selection OK."
display(widgets.interactive(f, w1=data_files_widget, w2=columns_widget, w3=status_widget))


#### Peak counting over selected time range.
Use the interactive sliders to set the start and finish times.

In [ ]:
#global variables
istart = 0
idone = 0
idx = []

with open("../results/" + data_file, 'r', encoding='utf-8-sig') as f: 
  A0 = np.transpose(np.genfromtxt(f, dtype=float, delimiter=','))
wtmax = np.ceil(A0[0,-1])
dmin = np.amin(A0[data_col])
dmax = np.amax(A0[data_col])
count_start = 0.0   # count start time
count_done = wtmax  # count end time
p = 4000            # desired length of resampled data

X0 = A0[0]
Y0 = A0[data_col]
tmin = np.min(X0)            # start time
tmax = np.max(X0)            # finish time
Y1 = (Y0-dmin) / (dmax-dmin) # data axis, normalized to range(0, 1.0)
sr = p / (tmax-tmin)         # the sample rate

# resample
f = interp1d(X0, Y1, kind='cubic')                    # define the resampling function
X = np.linspace(tmin, tmax, p+1, endpoint=True)       # define the new time steps
Y = f(X)                                              # resample the original signal

# apply high-pass filter to eliminate the stimulation "bump" in the data
sos = signal.butter(3, 0.1, btype='highpass', fs=sr, output='sos')
Yf = signal.sosfiltfilt(sos, Y) # zero phase shift filter

# apply low-pass filter to smooth out higher frequencies in the data
sos = signal.butter(7, 2.0, btype='lowpass', fs=sr, output='sos')
Yf = signal.sosfiltfilt(sos, Yf) # zero phase shift filter

pks,_ = signal.find_peaks(Yf,prominence=0.04)       # find indices of peaks in the resampled, filtered data
pidx = np.around((X0.shape[0]-1)*pks/p).astype(int) # convert to indices in the original data
pts = [A0[0][pidx], A0[data_col][pidx]]             # save the peaks as points in the original data

plt.close('all')
fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(12,6), constrained_layout=True)

@interact(
  cs = widgets.FloatSlider(value=count_start, min=0.0, max=wtmax-1.0, step=1.0,
                    description='start time', layout=Layout(width='700px')),
  cd = widgets.FloatSlider(value=count_done, min=1.0, max=wtmax, step=1.0,
                    description='finish time', layout=Layout(width='700px'))
)

def f(cs,cd):
  global istart, idone, idx
  istart = np.where(A0[0] > cs)[0][0]
  idone = np.where(A0[0] < cd)[0][-1]

  ax[0].cla()
  ax[0].set_title("Time range selection")
  ax[0].set(ylabel="intensity")
  ax[0].set(xlabel="time (s)")
  ax[0].plot(A0[0],A0[data_col])
  ax[0].plot([cs,cs],[dmin,dmax],color='green')
  ax[0].plot([cd,cd],[dmin,dmax],color='red')

  ax[1].cla()
  ax[1].set_title("Peaks in time range")
  ax[1].set(ylabel="intensity")
  ax[1].set(xlabel="time (s)")

  idx = pidx[(pidx >= istart) & (pidx <= idone)]
  ax[1].plot(A0[0,istart:idone],A0[data_col,istart:idone])
  ax[1].plot(A0[0,idx],A0[data_col,idx],'k.')                                # plot the peak locations
    
  plt.show()
  return("peaks count: " + str(len(idx)))

#### Save peaks to file.

In [ ]:
import csv

# save peaks to CSV file
with open ("../results/peaks.csv", 'w') as file:
  writer = csv.writer(file)
  for row in np.transpose(A0[:,idx]):
    writer.writerow([row[0], row[data_col]])
